In [23]:
from simtk.openmm import app
import simtk.openmm as mm
from simtk import unit
pdb = app.PDBFile('ethane.pdb')
forcefield = app.ForceField('ethane.gaff2.xml')
system = forcefield.createSystem(pdb.topology, nonbondedMethod=app.NoCutoff, constraints=app.HBonds)
integrator = mm.LangevinIntegrator(298.15*unit.kelvin, 5.0/unit.picoseconds, 2.0*unit.femtoseconds)
integrator.setConstraintTolerance(1e-5)
platform = mm.Platform.getPlatformByName('Reference')
simulation = app.Simulation(pdb.topology, system, integrator, platform)
simulation.context.setPositions(pdb.positions)

In [24]:
print('Minimizing...')

st = simulation.context.getState(getPositions=True,getEnergy=True)
print(F"Potential energy before minimization is {st.getPotentialEnergy()}")

simulation.minimizeEnergy(maxIterations=100)

st = simulation.context.getState(getPositions=True,getEnergy=True)
print(F"Potential energy after minimization is {st.getPotentialEnergy()}")

Minimizing...
Potential energy before minimization is 4.467818224810632 kJ/mol
Potential energy after minimization is 4.389967678922688 kJ/mol


In [25]:
from sys import stdout

print('Equilibrating...')

simulation.reporters.append(app.StateDataReporter(stdout, 100, step=True, 
    potentialEnergy=True, temperature=True, separator=','))
simulation.context.setVelocitiesToTemperature(150.0*unit.kelvin)
simulation.step(2500)

Equilibrating...
#"Step","Potential Energy (kJ/mole)","Temperature (K)"
100,10.485636927643162,182.04926427483915
200,18.79324224860203,260.97669891596746
300,17.822706918465922,265.72097772639427
400,18.29943344831253,416.88923385293293
500,28.481168590782758,491.69678505966573
600,16.31731513499158,341.7912799540316
700,12.999818555924424,294.92489316042014
800,21.280190734036246,352.0645965299952
900,31.32322879261704,193.72898442542933
1000,22.938038376254195,305.29639723179486
1100,10.695653343802238,273.8664676711135
1200,18.212659505158342,236.93783654540846
1300,16.20874226055811,180.7293610550918
1400,13.917486659673134,326.3988459204209
1500,23.49367113969725,371.3145304724672
1600,11.266318195390564,330.066607772836
1700,23.988026592806587,359.7825985373077
1800,14.521246855443675,321.93668996815916
1900,18.832856727694633,382.14894421209146
2000,35.16259215322299,295.2097640166719
2100,19.173218425254024,260.2698708921483
2200,13.637847782697506,266.63347911622253
2300,22.8

In [26]:
import time as time

print('Running Production')

# begin timer
tinit=time.time()

# clear simulation reporters
simulation.reporters.clear()

# reinitialize simulation reporters. Done because we want different information printed from the production run
# than the equilibriation run. 
# output basic simulation information below every 250,000 steps - 2 x 250,000 = 500,000fs or 500 ps
simulation.reporters.append(app.StateDataReporter(stdout, 250000, step=True, time=True, potentialEnergy=True,
                            temperature=True, speed=True, separator=','))

# write traj to a DCD file every 100 steps - 0.2ps
simulation.reporters.append(app.DCDReporter('ethane_sim.dcd', 100))

# run simulation for 1.0x10^7 steps - 20ns
simulation.step(10000000)

# end timer
tfinal=time.time()
print('Done!')
print('Time required for simulation:', tfinal-tinit, 'seconds')

Running Production
#"Step","Time (ps)","Potential Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
250000,500.0000000016593,33.17603828689481,469.9103783078711,0
500000,999.9999999901769,28.19311480241771,296.6876991144944,3.19e+04
750000,1499.9999999783536,28.861768550070046,330.84727381692375,3.12e+04
1000000,1999.9999999665301,18.370482235291185,385.4242272272084,3.15e+04
1250000,2499.9999999547067,23.364981426442856,109.6287929705714,3.18e+04
1500000,2999.9999999428833,13.483773642688366,353.6593473233898,3.19e+04
1750000,3499.99999993106,17.433633415828247,206.6215606104663,3.2e+04
2000000,3999.9999999192364,29.00135835631954,243.68775990446574,3.21e+04
2250000,4499.9999999992715,26.850370138240912,284.7271176978858,3.22e+04
2500000,5000.000000101135,19.206747845884554,389.4044541629241,3.21e+04
2750000,5500.000000202998,13.978203060936707,357.19618636516185,3.2e+04
3000000,6000.000000304862,13.912241567434599,519.8493807522063,3.2e+04
3250000,6500.000000406725,19.737312034851